# Classiq Coding Competition Spring 2022
### Fifth Place MCX Submission by Jan Tulowiecki

In [23]:
from cmath import phase

from qiskit import Aer, QuantumCircuit, QuantumRegister
from qiskit.circuit.library import RC3XGate
from qiskit.converters import circuit_to_instruction
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import CXCancellation, Optimize1qGates, Unroller


def apply_rc4x(qc: QuantumCircuit, q, inv=False):
    rc4x_gate = RC3XGate()#.control(1)
    # if inv:
    #    rc4x_gate = rc4x_gate.inverse()
    qc.append(rc4x_gate, q[0:-1])


def large_tofolli():
    ctrl = QuantumRegister(14)
    tar = QuantumRegister(1)
    anc = QuantumRegister(5)
    qc = QuantumCircuit(ctrl, tar, anc)

    qc.rcccx(ctrl[0], ctrl[1], ctrl[2], anc[0])
    qc.rcccx(ctrl[3], ctrl[4], ctrl[5], anc[1])
    qc.rcccx(ctrl[6], ctrl[7], ctrl[8], anc[2])
    qc.rcccx(ctrl[9], ctrl[10], ctrl[11], anc[3])
    qc.rcccx(ctrl[12], ctrl[13], anc[0], anc[4])

    qc.mcx([anc[1], anc[2], anc[3], anc[4]], tar)

    qc.append(RC3XGate().inverse(), [ctrl[12], ctrl[13], anc[0], anc[4]])
    qc.append(RC3XGate().inverse(), [ctrl[9], ctrl[10], ctrl[11], anc[3]])
    qc.append(RC3XGate().inverse(), [ctrl[6], ctrl[7], ctrl[8], anc[2]])
    qc.append(RC3XGate().inverse(), [ctrl[3], ctrl[4], ctrl[5], anc[1]])
    qc.append(RC3XGate().inverse(), [ctrl[0], ctrl[1], ctrl[2], anc[0]])
    return qc


def unroll_circ(circuit):
    pass_ = Unroller(['u1', 'u2', 'u3', 'cx'])
    pm = PassManager(pass_)
    unrolled = pm.run(circuit)
    unrolled_2 = PassManager([Optimize1qGates(), CXCancellation()]).run(unrolled)
    return unrolled_2


def validate():
    for f in [0, 1]:
        ctrl = QuantumRegister(14)
        tar = QuantumRegister(1)
        anc = QuantumRegister(5)
        qc = QuantumCircuit(ctrl, tar, anc)

        tofolli_circ = unroll_circ(large_tofolli())
        if f == 0:
            print(tofolli_circ.count_ops())
            print(f'depth = {tofolli_circ.depth()}')
            print(tofolli_circ.qasm())
        tofolli_instr = circuit_to_instruction(tofolli_circ)

        qubits = [q for q in ctrl] + [q for q in tar] + [q for q in anc]
        qc.h(ctrl)
        if f == 1:
            qc.x(tar)
        qc.append(tofolli_instr, qubits)
        qc_unrolled = unroll_circ(qc)
        
        sim_statevector = Aer.get_backend('statevector_simulator')
        # sim_statevector.set_options(precision='single')
        result = sim_statevector.run(qc_unrolled).result()
        statevector = result.get_statevector(qc_unrolled)

        amplitude = 2 ** -7
        for i, v in enumerate(statevector):
            mask = f'{i:#022b}'
            expected_ampl = 0.0
            if mask.startswith('0b00000'):
                # ancillas are cleared
                if mask.endswith('1' * 14):
                    if mask[7] == str(1 - f):
                        # this state should be flipped!
                        expected_ampl = amplitude
                elif mask[7] == str(f):
                    # this state should NOT be flipped!
                    expected_ampl = amplitude
            if abs(expected_ampl - abs(v)) > 1e-5 or (expected_ampl > 1e-4 and abs(phase(v)) > 1e-4):
                print(f'[ERR] Mask {mask} has incorrect value of {v} for {f=}')
                return 

if __name__ == '__main__':
    validate()

OrderedDict([('cx', 96), ('u1', 60), ('u2', 48)])
depth = 101
OPENQASM 2.0;
include "qelib1.inc";
qreg q349[14];
qreg q350[1];
qreg q351[5];
u2(0,pi) q350[0];
u2(pi/4,pi) q351[0];
cx q349[2],q351[0];
u2(0,3*pi/4) q351[0];
cx q349[0],q351[0];
u1(pi/4) q351[0];
cx q349[1],q351[0];
u1(-pi/4) q351[0];
cx q349[0],q351[0];
u1(pi/4) q351[0];
cx q349[1],q351[0];
u2(pi/4,3*pi/4) q351[0];
cx q349[2],q351[0];
u2(0,3*pi/4) q351[0];
u2(pi/4,pi) q351[1];
cx q349[5],q351[1];
u2(0,3*pi/4) q351[1];
cx q349[3],q351[1];
u1(pi/4) q351[1];
cx q349[4],q351[1];
u1(-pi/4) q351[1];
cx q349[3],q351[1];
u1(pi/4) q351[1];
cx q349[4],q351[1];
u2(pi/4,3*pi/4) q351[1];
cx q349[5],q351[1];
u2(0,3*pi/4) q351[1];
u2(pi/4,pi) q351[2];
cx q349[8],q351[2];
u2(0,3*pi/4) q351[2];
cx q349[6],q351[2];
u1(pi/4) q351[2];
cx q349[7],q351[2];
u1(-pi/4) q351[2];
cx q349[6],q351[2];
u1(pi/4) q351[2];
cx q349[7],q351[2];
u2(pi/4,3*pi/4) q351[2];
cx q349[8],q351[2];
u2(0,3*pi/4) q351[2];
u2(pi/4,pi) q351[3];
cx q349[11],q351[3];
u2(0

/tmp/ipykernel_738/498575263.py:88: DeprecationWarning: The return type of saved statevectors has been changed from a `numpy.ndarray` to a `qiskit.quantum_info.Statevector` as of qiskit-aer 0.10. Accessing numpy array attributes is deprecated and will result in an error in a future release. To continue using saved result objects as arrays you can explicitly cast them using  `np.asarray(object)`.
  for i, v in enumerate(statevector):


         ┌───┐┌─────────────────┐
 q467_0: ┤ H ├┤0                ├
         ├───┤│                 │
 q467_1: ┤ H ├┤1                ├
         ├───┤│                 │
 q467_2: ┤ H ├┤2                ├
         ├───┤│                 │
 q467_3: ┤ H ├┤3                ├
         ├───┤│                 │
 q467_4: ┤ H ├┤4                ├
         ├───┤│                 │
 q467_5: ┤ H ├┤5                ├
         ├───┤│                 │
 q467_6: ┤ H ├┤6                ├
         ├───┤│                 │
 q467_7: ┤ H ├┤7                ├
         ├───┤│                 │
 q467_8: ┤ H ├┤8                ├
         ├───┤│                 │
 q467_9: ┤ H ├┤9                ├
         ├───┤│   circuit-14438 │
q467_10: ┤ H ├┤10               ├
         ├───┤│                 │
q467_11: ┤ H ├┤11               ├
         ├───┤│                 │
q467_12: ┤ H ├┤12               ├
         ├───┤│                 │
q467_13: ┤ H ├┤13               ├
         ├───┤│                 │
   q468: ┤ X ├